## FAERS natural products sensitivity analysis



In [1]:
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, clear_output
import re

In [ ]:
##read all unmapped strings and remove np reference set strings and negative pairs

In [2]:
df = pd.read_csv('data/upper_unmap_orig_drug_names_202201201812.csv')
dfpos = pd.read_csv('data/NP_FAERS_mapped_20220215.csv')
dfneg = pd.read_csv('data/NP_FAERS_negative_pairs_20220222.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569525 entries, 0 to 569524
Data columns (total 1 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   drug_name_original  412264 non-null  object
dtypes: object(1)
memory usage: 4.3+ MB


In [4]:
dfpos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5358 entries, 0 to 5357
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   FAERS_drug_match  5358 non-null   object
 1   lookup_value      5358 non-null   object
dtypes: object(2)
memory usage: 83.8+ KB


In [5]:
dfneg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9242 entries, 0 to 9241
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   FAERS_drug_match  9242 non-null   object
 1   lookup_value      9242 non-null   object
dtypes: object(2)
memory usage: 144.5+ KB


In [6]:
dfpos = dfpos.drop(['lookup_value'], axis=1)
dfpos = dfpos.drop_duplicates(ignore_index=True)
dfpos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2630 entries, 0 to 2629
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   FAERS_drug_match  2630 non-null   object
dtypes: object(1)
memory usage: 20.7+ KB


In [7]:
dfneg = dfneg.drop(['lookup_value'], axis=1)
dfneg = dfneg.drop_duplicates(ignore_index=True)
dfneg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4465 entries, 0 to 4464
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   FAERS_drug_match  4465 non-null   object
dtypes: object(1)
memory usage: 35.0+ KB


In [2]:
def clean(text):
    #remove all non-ascii, special characters and keep alphabets and space only. Can also use isalpha()
    #convert to uppercase
    #remove extra spaces
    regex = re.compile('[^a-zA-Z ]')
    r = regex.sub('', text)
    result = re.sub(' +', ' ', r)
    result = result.strip()
    return result.upper()

In [13]:
df.loc[df['drug_name_original'].isna()]

,drug_name_original
276498,NaN
339826,NaN
358758,NaN
367030,NaN
367031,NaN
...,...
524283,NaN
524284,NaN
524285,NaN
524286,NaN


In [14]:
#drop NA values
df = df.dropna()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412264 entries, 0 to 412263
Data columns (total 1 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   drug_name_original  412264 non-null  object
dtypes: object(1)
memory usage: 3.1+ MB


In [16]:
df = df.reset_index(drop=True)

In [18]:
df = df.rename(columns={'drug_name_original':'FAERS_drug_match'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412264 entries, 0 to 412263
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   FAERS_drug_match  412264 non-null  object
dtypes: object(1)
memory usage: 3.1+ MB


In [21]:
dftemp = pd.merge(df,dfpos, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
dftemp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409635 entries, 0 to 412263
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   FAERS_drug_match  409635 non-null  object
dtypes: object(1)
memory usage: 6.3+ MB


In [22]:
dfunmap = pd.merge(dftemp,dfneg, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
dfunmap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405332 entries, 0 to 409634
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   FAERS_drug_match  405332 non-null  object
dtypes: object(1)
memory usage: 6.2+ MB


In [23]:
dfunmap = dfunmap.reset_index(drop=True)
dfunmap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405332 entries, 0 to 405331
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   FAERS_drug_match  405332 non-null  object
dtypes: object(1)
memory usage: 3.1+ MB


In [24]:
dfunmap.to_csv('data/upper_unmap_orig_drug_names_no_model_overlap_20220224.csv', index=False)

In [25]:
dfunmap.head()

,FAERS_drug_match
0,ACRINOL HYDRATE
1,MAXAZIDE/TRIAMTEREN HCTZ
2,KETOBEMIDONE (KETOGAN) (KETOGIN)
3,TYLENOL XTRA STRENGTH
4,MINOCYCLINE(MINOCYLCINE HYDROCHLORIDE)


### Read unmapped data from file and load tensorflow model for prediction

In [2]:
import tensorflow as tf
import string
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [50]:
# how i add noise 
# string w 
# proportion of noise added 
# uniform random from [0,1]
# if <1/3 edit one position with new random character, else if <2/3 delete one position, else add one random character 
def add_noise(w, percent):
  ''' edit, del, add'''
  positions = random.choices(range(len(w)), k=int(percent*len(w)))
  for p in positions:
    r = random.random()
    if r <= 0.3333: # edit
      w = w[:p] + random.choice(string.ascii_uppercase) + w[p+1:]
    elif r<= 0.6667: # delete
      w = w[:p] + w[p+1:]
    elif r<=1: # add
      w = w[:p] + random.choice(string.ascii_uppercase) + w[p:]
  return w

def clean(text):
    #remove all non-ascii, special characters and keep alphabets and space only. Can also use isalpha()
    #convert to uppercase
    #remove extra spaces
    regex = re.compile('[^a-zA-Z ]')
    r = regex.sub('', text)
    result = re.sub(' +', ' ', r)
    result = result.strip()
    return result.upper()
def clean_dataset(data):
  x = []
  y = []
  for i in range(data.shape[0]):
    w = clean(data.FAERS_drug_match.iloc[i])
    x.append(w)
  return x

def encode_dataset(x):
  encode_dict = {l:i+1 for i,l in enumerate(string.ascii_uppercase + " ")}
  Xtrain = [[encode_dict[m] for m in n] for n in x]
  return Xtrain

def clean_encode_padding(q, maxlen):
  q = clean(q)
  encode_dict = {l:i+1 for i,l in enumerate(string.ascii_uppercase + " ")}
  return tf.keras.preprocessing.sequence.pad_sequences(
    [encode_dict[m] for m in q] , padding="post", maxlen=maxlen)

def padding_dataset(X,maxlen):
  padded_x = tf.keras.preprocessing.sequence.pad_sequences(
      X, padding="post", maxlen=maxlen)
  return padded_x

def cosine_distance(vects):
    x, y = vects
    return 1-tf.reduce_sum(tf.multiply(x,y),axis=1, keepdims=True)/(tf.norm(x,axis=1,keepdims=True)*tf.norm(y,axis=1,keepdims=True))

def loss(margin=1):
    def contrastive_loss(y_true, y_pred):
        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss


In [5]:
maxlen = 400

In [6]:
def build_model2(model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin):
    input_x = tf.keras.layers.Input(maxlen)
    input_1 = tf.keras.layers.Input(maxlen)
    input_2 = tf.keras.layers.Input(maxlen)
    embedding = tf.keras.layers.Embedding(input_dim=28, output_dim=embedding_dim, mask_zero=True)
    x = embedding(input_x)
    
    if model_type == "lstm":
        x = tf.keras.layers.LSTM(num_rnn_node)(x)
    elif model_type=="gru":
        x = tf.keras.layers.GRU(num_rnn_node)(x)
 
    num = num_dense_node
    for _ in range(num_layer):
        x = tf.keras.layers.Dense(num, activation=activation_fn)(x)
        num /= 2
        
    embedding_network = tf.keras.Model(input_x, x)

    tower_1 = embedding_network(input_1)
    tower_2 = embedding_network(input_2)

    merge_layer = tf.keras.layers.Lambda(cosine_distance)([tower_1, tower_2])
    output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(merge_layer)
    contr = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)
    
    if optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer =="RMSprop":                
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    contr.compile(loss=loss(margin= margin), optimizer=opt, metrics=["accuracy"])
    return contr

model = build_model2("lstm", 256, 512, 256, 1, "tanh", 2e-4, "Adam", 0.8)

2022-03-03 10:24:40.663016: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-03 10:24:41.613205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9648 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5


In [7]:
model.load_weights("exp3/alstm-22-0.01.hdf5")

In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 model (Functional)             (None, 256)          1713408     ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

In [9]:
for layer in model.layers:
    print(layer)

In [23]:
len(model.get_weights())

8

In [24]:
###read unmapped data
dfunmap = pd.read_csv('data/upper_unmap_orig_drug_names_no_model_overlap_20220224.csv')
dfunmap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405332 entries, 0 to 405331
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   FAERS_drug_match  405332 non-null  object
dtypes: object(1)
memory usage: 3.1+ MB


In [28]:
df = dfunmap[0:1000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   FAERS_drug_match  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [29]:
df.head()

,FAERS_drug_match
0,ACRINOL HYDRATE
1,MAXAZIDE/TRIAMTEREN HCTZ
2,KETOBEMIDONE (KETOGAN) (KETOGIN)
3,TYLENOL XTRA STRENGTH
4,MINOCYCLINE(MINOCYLCINE HYDROCHLORIDE)


In [30]:
from datetime import datetime
datetime.now()

datetime.datetime(2022, 3, 3, 10, 35, 54, 447430)

In [71]:
##read lb to common names and get unique names, also get unique names of drugs
##read lb to common names and select random NPs for computing similarity with unmapped drugs
np_list = ['ABSINTHE',
'ABSINTHE WORMWOOD',
'ABSINTHIUM',
'ABSINTHIUM OFFICINALE WHOLE',
'ABSINTHIUM VULGARE WHOLE',
'ABSINTHIUM WHOLE',
'ABSINTH SAGEWORT WHOLE',
'ABSINTH WORMWOOD WHOLE',
'PRUNUS ARMENIACA', 'APRICOT', 'APIUM GRAVEOLENS', 'CELERY']

In [72]:
##df has first 1000 of unmapped strings
test_unmap = clean_dataset(df)
test_unmap_encode = encode_dataset(test_unmap)
padded_test = padding_dataset(test_unmap_encode,maxlen)

In [73]:
type(padded_testt[0,:])

numpy.ndarray

In [74]:
encode_dict = {l:i+1 for i,l in enumerate(string.ascii_uppercase + " ")}

In [76]:
dUnique_seq_list = [[encode_dict[m] for m in n] for n in np_list]
dUnique = tf.keras.preprocessing.sequence.pad_sequences(
    dUnique_seq_list, padding="post", maxlen=maxlen)

In [77]:
len(dUnique)

12

In [78]:
dUnique.shape[0]

12

In [82]:
print(datetime.now())
predicts = model.predict([np.tile(padded_test[0,:], (dUnique.shape[0],1)), dUnique])
print(datetime.now())

2022-03-03 11:02:04.597646
2022-03-03 11:02:04.711359


In [83]:
predicts

array([[0.0667495 ],
       [0.01420481],
       [0.00045384],
       [0.00191143],
       [0.00047604],
       [0.00043126],
       [0.00109308],
       [0.00138434],
       [0.02856903],
       [0.06765651],
       [0.00571964],
       [0.00591195]], dtype=float32)

In [ ]:
##takes 4 seconds with 12 items in dUnique

In [ ]:
test_eval = pd.DataFrame(columns = ['FAERS_drug_match', ])

In [ ]:
#find ranks 1, 2 and 3 (can go upto any number n) from the predicted similarities for the test data
for i in range(padded_testt.shape[0]):
    predicts = model.predict([np.tile(padded_testt[i,:], (dUnique.shape[0],1)), dUnique])
    argsort = np.argsort(-predicts.flatten())
    test_eval.at[i, 'FAERS_drug_match'] = test.at[i, 'FAERS_drug_match']
    
test_eval.head()

In [98]:
np_df = pd.read_csv('data/np_names_clean_202201201810.csv')
np_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6195 entries, 0 to 6194
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   concept_id  6195 non-null   int64 
 1   np_name     6195 non-null   object
dtypes: int64(1), object(1)
memory usage: 96.9+ KB


In [99]:
np_df.head()

,concept_id,np_name
0,-7991903,SUO CAO
1,-7993765,PURPLE NUT SEDGE WHOLE
2,-7997759,POGOSTEMON JAVANICUS WHOLE
3,-7992037,HORDEUM AMERICANUM WHOLE
4,-7992977,SAMAMA CITRIFOLIA WHOLE


In [100]:
np_df['np_name_clean'] = np_df['np_name'].apply(clean)

In [101]:
np_df.head()

,concept_id,np_name,np_name_clean
0,-7991903,SUO CAO,SUO CAO
1,-7993765,PURPLE NUT SEDGE WHOLE,PURPLE NUT SEDGE WHOLE
2,-7997759,POGOSTEMON JAVANICUS WHOLE,POGOSTEMON JAVANICUS WHOLE
3,-7992037,HORDEUM AMERICANUM WHOLE,HORDEUM AMERICANUM WHOLE
4,-7992977,SAMAMA CITRIFOLIA WHOLE,SAMAMA CITRIFOLIA WHOLE


In [102]:
dUnique_df = pd.DataFrame(columns = ['dUnique_label','dUnique_seq', 'dUnique_seq_padded'])
dUnique_df['dUnique_label'] = np.unique(np_df.np_name_clean.tolist())
dUnique_df.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,AARONS ROD WHOLE,NaN,NaN
1,AARONSROD,NaN,NaN
2,ABIES ABIES WHOLE,NaN,NaN
3,ABIES ALBA,NaN,NaN
4,ABIES ALBA MILL,NaN,NaN


In [103]:
dUnique_seq_list = [[encode_dict[m] for m in n] for n in dUnique_df['dUnique_label'].tolist()]
len(dUnique_seq_list)

6160

In [104]:
for i in range(len(dUnique_df.index)):
    dUnique_df.at[i, 'dUnique_seq'] = np.array(dUnique_seq_list[i])
dUnique_df.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,AARONS ROD WHOLE,"[1, 1, 18, 15, 14, 19, 27, 18, 15, 4, 27, 23, ...",NaN
1,AARONSROD,"[1, 1, 18, 15, 14, 19, 18, 15, 4]",NaN
2,ABIES ABIES WHOLE,"[1, 2, 9, 5, 19, 27, 1, 2, 9, 5, 19, 27, 23, 8...",NaN
3,ABIES ALBA,"[1, 2, 9, 5, 19, 27, 1, 12, 2, 1]",NaN
4,ABIES ALBA MILL,"[1, 2, 9, 5, 19, 27, 1, 12, 2, 1, 27, 13, 9, 1...",NaN


In [105]:
dUnique = tf.keras.preprocessing.sequence.pad_sequences(
    list(dUnique_df['dUnique_seq']), padding="post", maxlen=maxlen)

In [106]:
for i in range(len(dUnique_df.index)):
    dUnique_df.at[i, 'dUnique_seq_padded'] = dUnique[i]
dUnique_df.head()

,dUnique_label,dUnique_seq,dUnique_seq_padded
0,AARONS ROD WHOLE,"[1, 1, 18, 15, 14, 19, 27, 18, 15, 4, 27, 23, ...","[1, 1, 18, 15, 14, 19, 27, 18, 15, 4, 27, 23, ..."
1,AARONSROD,"[1, 1, 18, 15, 14, 19, 18, 15, 4]","[1, 1, 18, 15, 14, 19, 18, 15, 4, 0, 0, 0, 0, ..."
2,ABIES ABIES WHOLE,"[1, 2, 9, 5, 19, 27, 1, 2, 9, 5, 19, 27, 23, 8...","[1, 2, 9, 5, 19, 27, 1, 2, 9, 5, 19, 27, 23, 8..."
3,ABIES ALBA,"[1, 2, 9, 5, 19, 27, 1, 12, 2, 1]","[1, 2, 9, 5, 19, 27, 1, 12, 2, 1, 0, 0, 0, 0, ..."
4,ABIES ALBA MILL,"[1, 2, 9, 5, 19, 27, 1, 12, 2, 1, 27, 13, 9, 1...","[1, 2, 9, 5, 19, 27, 1, 12, 2, 1, 27, 13, 9, 1..."


In [107]:
len(dUnique)

6160

In [108]:
print(datetime.now())
predicts = model.predict([np.tile(padded_test[0,:], (dUnique.shape[0],1)), dUnique])
print(datetime.now())

2022-03-03 11:11:23.993556
2022-03-03 11:11:25.176727


In [109]:
predicts

array([[0.00963356],
       [0.01632481],
       [0.00637359],
       ...,
       [0.00602851],
       [0.03520529],
       [0.00332402]], dtype=float32)

In [113]:
pred_sort = np.sort(predicts)
pred_sort[-1]

array([0.00332402], dtype=float32)

In [115]:
pred_sort[-1]

array([0.00332402], dtype=float32)

In [110]:
argsort = np.argsort(-predicts.flatten())

In [111]:
argsort[0]

3906

In [116]:
dUnique_df.iloc[argsort[0]].dUnique_label

'MITRAGYNA SPECIOSA WHOLE'